In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csc_matrix, csr_matrix
from sklearn.metrics import pairwise_distances

In [2]:
anime = pd.read_csv("anime.csv")
anime.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


In [3]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17562 entries, 0 to 17561
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MAL_ID         17562 non-null  int64 
 1   Name           17562 non-null  object
 2   Score          17562 non-null  object
 3   Genres         17562 non-null  object
 4   English name   17562 non-null  object
 5   Japanese name  17562 non-null  object
 6   Type           17562 non-null  object
 7   Episodes       17562 non-null  object
 8   Aired          17562 non-null  object
 9   Premiered      17562 non-null  object
 10  Producers      17562 non-null  object
 11  Licensors      17562 non-null  object
 12  Studios        17562 non-null  object
 13  Source         17562 non-null  object
 14  Duration       17562 non-null  object
 15  Rating         17562 non-null  object
 16  Ranked         17562 non-null  object
 17  Popularity     17562 non-null  int64 
 18  Members        17562 non-n

In [4]:
rating = pd.read_csv("rating_complete.csv")
rating.head()

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9


In [5]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57633278 entries, 0 to 57633277
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 1.3 GB


In [6]:
# adicionando novos dados

def entry():
    
    """cria um user_id para as novas avaliações,
    recebe os nomes e avaliações dos animes através de inputs,
    busca as respectivas anime_ids,
    e armazena as informações na tabela new_rating,
    enquanto houver novas entradas"""
    
    global a
    new_rating["user_id"][a]=-1
    anime_name = input("Nome do anime: ")
    new_rating["anime_id"][a]=int(anime.loc[anime["Name"]==anime_name,"MAL_ID"])
    new_rating["rating"][a]=int(input("Sua nota para o anime: "))
    mais = input("Quer adicionar avaliar mais animes?(s/n) ")
    while mais != "s" and mais != "n": 
        mais = input("Quer adicionar avaliar mais animes?(s/n) ")
    while mais == "s":
        a+=1
        return entrada()

In [7]:
# tabela para novos dados

#new_rating = pd.DataFrame(index=range(17562),columns=["user_id","anime_id","rating"])
#a=0
#entry()

In [8]:
# tabelas-exemplo

#new_rating = pd.read_csv("bets_favs.csv")
new_rating = pd.read_csv("pigs_favs.csv")

In [9]:
new_rating.head()

,Unnamed: 0,user_id,anime_id,rating
0,0,-1,5114,10
1,1,-1,21,10
2,2,-1,223,9
3,3,-1,11061,10
4,4,-1,392,9


In [10]:
# juntando na original pra fazer a esparsa

merged_rating = pd.concat([new_rating.dropna(),rating])

In [11]:
# criando a esparsa

user_id = list(sorted(merged_rating["user_id"].unique()))
anime_id = list(sorted(merged_rating["anime_id"].unique()))

data = merged_rating["rating"].tolist()
row = merged_rating["user_id"].astype("category").cat.codes
col = merged_rating["anime_id"].astype("category").cat.codes
sparse_pivot = csc_matrix((data, (row, col)), shape=(len(user_id), len(anime_id)))

In [12]:
# identificando a semelhança entre usuário novo e os da tabela original

similarities = pairwise_distances(sparse_pivot[1:],sparse_pivot[0:1])

In [13]:
similarities

array([[56.08921465],
       [83.28865469],
       [62.72160712],
       ...,
       [48.97958759],
       [76.88302804],
       [49.74937186]])

In [14]:
# separando os 100 mais próximos

pd_similarities = pd.DataFrame(1/(1+similarities), index=user_id[1:], columns=user_id[0:1]).sort_values(by=-1,ascending=False).head(100)

In [15]:
pd_similarities

,-1
247845,0.033143
332879,0.033143
350265,0.033012
54555,0.032938
83884,0.032810
...,...
264101,0.030639
156051,0.030624
98803,0.030624
350927,0.030624


In [16]:
# separando as avaliações dos mais próximos

recommendations = rating.loc[rating["user_id"].isin(pd_similarities.index)]

In [17]:
recommendations

,user_id,anime_id,rating
125172,799,16498,9
125173,799,1535,9
125174,799,223,7
125175,799,11061,10
125176,799,20507,8
...,...,...,...
57129823,350265,2476,6
57129824,350265,7724,6
57129825,350265,16498,10
57234508,350927,1535,10


In [18]:
# pivotando pra deixar os usuários nas colunas e animes nas linhas

peers_ratings = recommendations.pivot_table(values="rating",index="anime_id",columns="user_id",aggfunc="mean",fill_value=np.nan)

In [19]:
peers_ratings

user_id,799,4679,16529,18646,27590,27726,39473,49559,54555,57418,...,321248,321683,321815,324661,329254,331538,332879,341032,350265,350927
anime_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,9.0,NaN,NaN,9.0,NaN,8.0,NaN,8.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# atribuindo peso pras notas de acordo com a proximidade

for user_id, factor in dict(pd_similarities[-1]).items():
    peers_ratings[user_id] = (peers_ratings[user_id]**2) * (factor**2)

In [21]:
peers_ratings

user_id,799,4679,16529,18646,27590,27726,39473,49559,54555,57418,...,321248,321683,321815,324661,329254,331538,332879,341032,350265,350927
anime_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,0.083125,NaN,NaN,0.080564,NaN,0.066597,NaN,0.069435,0.061084,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,NaN,NaN,NaN,NaN,0.035806,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.095632,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# ranking de recomendações

total_rating = pd.DataFrame(peers_ratings.T.sum().sort_values(ascending=False)).rename(columns={0:"score"})

In [23]:
total_rating

,score
anime_id,
1535,5.810683
5114,5.456621
16498,3.310871
11061,2.815907
20,2.389383
...,...
482,0.015914
30016,0.015849
31710,0.015849


In [24]:
# descartando animes já avaliados pelo novo usuário e pegando o top10

top10_id = total_rating.loc[~total_rating.index.isin(new_rating["anime_id"])].head(10)

In [25]:
top10_id

,score
anime_id,
813,0.418449
1575,0.253003
9253,0.252057
30276,0.250141
21881,0.204078
28171,0.141385
22535,0.140531
9919,0.137491
25777,0.119909


In [26]:
# associando o id ao nome do anime

top10_name = anime.loc[anime["MAL_ID"].isin(top10_id.index),["MAL_ID","Name"]].rename(columns={"MAL_ID":"anime_id","Name":"anime_name"}).set_index("anime_id")

In [27]:
top10_name

,anime_name
anime_id,
121,Fullmetal Alchemist
813,Dragon Ball Z
1575,Code Geass: Hangyaku no Lelouch
9253,Steins;Gate
9919,Ao no Exorcist
21881,Sword Art Online II
22535,Kiseijuu: Sei no Kakuritsu
25777,Shingeki no Kyojin Season 2
28171,Shokugeki no Souma


In [28]:
# o resultado

top10 = pd.merge(left=top10_id,right=top10_name,left_index=True,right_index=True).sort_values(by="score",ascending=False)

In [29]:
top10

,score,anime_name
anime_id,,
813,0.418449,Dragon Ball Z
1575,0.253003,Code Geass: Hangyaku no Lelouch
9253,0.252057,Steins;Gate
30276,0.250141,One Punch Man
21881,0.204078,Sword Art Online II
28171,0.141385,Shokugeki no Souma
22535,0.140531,Kiseijuu: Sei no Kakuritsu
9919,0.137491,Ao no Exorcist
25777,0.119909,Shingeki no Kyojin Season 2
